# Function Decorators and Closures

## Decorators 101

A decorator is a callable that takes another function as argument. For example, the following snippet
```
@decorate
def target():
    print('running target()')
```
is equivalent to
```
def target():
    print('running target()')

target = decorate(target)
```

They are executed immediately when a module is loaded.

In [ ]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

target()
print('target:', target)

## When Python Executes Decorators
They run right after decorated function is defined, which is usually at _import time_. Example [_registration.py_](./registration.py).

In [ ]:
%run registration.py

If _registration.py_ is imported.

In [ ]:
import registration
print('registry ->', registry)

## Variable Scope Rules

If a variable is neither a parameter nor locally assigned, then the interpreter treats it as a global variable.

In [ ]:
def f1(a):
    print(a)
    print(b)
    
f1(3)

define global variable `b`

In [ ]:
b = 6
f1(3)

A global variable can be changed to a local variable in a function.

In [ ]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9
    
f2(3)

To make `b` always a global variable. Use the keyword `global`.

In [ ]:
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)
print(b)
b = 30
f3(3)

## Closures

A closure is a function with an extended scope that encompasses nonglobal variables referenced in the body of the function but not defined there.

def make_averager():
    series = []
    count = 0
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

In [16]:
avg = make_averager()
print(avg(10), avg(11), avg(12))

10.0 10.5 11.0


In [17]:
print('avg local vars:', avg.__code__.co_varnames)
print('avg free vars:', avg.__code__.co_freevars)
print('avg closures:', avg.__closure__[0].cell_contents)

avg local vars: ('new_value', 'total')
avg free vars: ('series',)
avg closures: [10, 11, 12]


### The nonlocal Declaration

A broken example

In [18]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager

In [19]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

The reason is in line 5, `count` is a new variable instead of the free variable in the closure. Then the question is how to avoid immutable type variables be re-declared? `nonlocal`

In [20]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

In [21]:
avg = make_averager()
print(avg(10), avg(11), avg(12))

10.0 10.5 11.0


## Implementing a Simple Decorator

prints the elapsed time, the arguments passed and the result of the call.

In [25]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        arg_str = ', '.join(repr(arg) for arg in args)
        print('{:.8f}s {}({}) -> {}'.format(elapsed, func.__name__, arg_str, result))
        return result
    return clocked

In [26]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
0.12308071s snooze(0.123) -> None
**************************************** Calling factorial(6)
0.00000026s factorial(1) -> 1
0.00002216s factorial(2) -> 2
0.00003887s factorial(3) -> 6
0.00005429s factorial(4) -> 24
0.00007078s factorial(5) -> 120
0.00008778s factorial(6) -> 720
6! = 720


The shortcoming are
1. it does not support keyword arguments
1. `__name__` and `__doc__` are of the decorated function.

Use `functools.wraps` to improve

In [29]:
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
        return result
    return clocked        

In [30]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(n=6)')
print('6! =', factorial(n=6))

**************************************** Calling snooze(.123)
[0.12318730s] snooze(0.123) -> None 
**************************************** Calling factorial(n=6)
[0.00000024s] factorial(1) -> 1 
[0.00002456s] factorial(2) -> 2 
[0.00004244s] factorial(3) -> 6 
[0.00005817s] factorial(4) -> 24 
[0.00007439s] factorial(5) -> 120 
[0.00009203s] factorial(n=6) -> 720 
6! = 720


## Decorators in the Standard Library

* Built-in decorators: `property`, `classmethod`, `staticmethod`.
* functools `wraps, lru_cache, singledispatch`.

### Memoization with functools.lru_cache

Compute Fibonacci 6 without cache

In [32]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonaaci(6))

[0.00000072s] fibonacci(0) -> 0 
[0.00000167s] fibonacci(1) -> 1 
[0.00089574s] fibonacci(2) -> 1 
[0.00000048s] fibonacci(1) -> 1 
[0.00000072s] fibonacci(0) -> 0 
[0.00000072s] fibonacci(1) -> 1 
[0.00008559s] fibonacci(2) -> 1 
[0.00161481s] fibonacci(3) -> 2 
[0.00289488s] fibonacci(4) -> 3 
[0.00000072s] fibonacci(1) -> 1 
[0.00000095s] fibonacci(0) -> 0 
[0.00000072s] fibonacci(1) -> 1 
[0.00059700s] fibonacci(2) -> 1 
[0.00088811s] fibonacci(3) -> 2 
[0.00000072s] fibonacci(0) -> 0 
[0.00000095s] fibonacci(1) -> 1 
[0.00008345s] fibonacci(2) -> 1 
[0.00000048s] fibonacci(1) -> 1 
[0.00000072s] fibonacci(0) -> 0 
[0.00000119s] fibonacci(1) -> 1 
[0.00008106s] fibonacci(2) -> 1 
[0.00016236s] fibonacci(3) -> 2 
[0.00032568s] fibonacci(4) -> 3 
[0.00129509s] fibonacci(5) -> 5 
[0.00449252s] fibonaaci(6) -> 8 
8


Optimizaiton with an LRU cache

In [33]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonaaci(6))

[0.00000119s] fibonacci(0) -> 0 
[0.00000167s] fibonacci(1) -> 1 
[0.00254393s] fibonacci(2) -> 1 
[0.00000215s] fibonacci(3) -> 2 
[0.00320792s] fibonacci(4) -> 3 
[0.00000238s] fibonacci(5) -> 5 
[0.00449109s] fibonaaci(6) -> 8 
8


`lru_cache` has two optional parameters `functools.lru_cache(maxsize=128, typed=False)`. See [the doc](https://docs.python.org/3/library/functools.html#functools.lru_cache).

### Generic Functions with Single Dispatch

Customize HTML otuput based on the type of the input argument

In [34]:
from functools import singledispatch
from collections import abc
import numbers, html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [36]:
print(htmlize({1,2,3}))
print(htmlize(abs))
print(htmlize('Heimlich & Co.\n- a game'))
print(htmlize(42))
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<p>Heimlich &amp; Co.<br>
- a game</p>
<pre>42 (0x2a)</pre>
<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


## Stacked Decorators
```
@d1
@d2
...
@dn
def f():
  ...
```

Is the same as `f = d1(d2(...dn(f)))`

## Parameterized Decorators

In [37]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s {name}({args}) -> {result}]'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            args = ', '.join(repr(arg) for arg in _args)
            name = func.__name__
            result = repr(_result)
            print(fmt.format(**locals()))
            return result
        return clocked
    return decorate

In [38]:
@clock()
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

[0.12962103s snooze(0.123) -> None]
[0.12318158s snooze(0.123) -> None]
[0.12317586s snooze(0.123) -> None]


In [39]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze: 0.12317943572998047s
snooze: 0.12316775321960449s
snooze: 0.12314844131469727s


In [40]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.124s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
